Region traces were intially only done for H&E, CODEX, and Xenium. Here adding in Cosmx and Visium HD 

In [41]:
import pickle
import json
import os
import re
import sys
from copy import deepcopy
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import skimage
import tifffile
import yaml
from matplotlib.collections import PolyCollection
from einops import rearrange, repeat
from pydantic_extra_types.color import Color

In [42]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['svg.fonttype'] = 'none'

In [43]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
%autoreload 2

In [45]:
from mushroom.mushroom import Mushroom, DEFAULT_CONFIG
import mushroom.utils as utils
import mushroom.visualization.utils as vis_utils
import mushroom.data.multiplex as multiplex
import mushroom.data.visium as visium
import mushroom.data.xenium as xenium
import mushroom.data.cosmx as cosmx
import mushroom.visualization.tiling_utils as tiling_utils

In [46]:
source_root = '/diskmnt/Projects/Users/estorrs/mushroom/data'
target_root = '/data/estorrs/mushroom/data'

In [47]:
run_dir = '/data/estorrs/mushroom/data/projects/submission_v1'


In [48]:
def alter_filesystem(config, source_root, target_root):
    for entry in config['sections']:
        for mapping in entry['data']:
            mapping['filepath'] = mapping['filepath'].replace(source_root, target_root)
        
    return config

In [49]:
# case = 'HT891Z1'
# case = 'HT704B1'
case = 'HT206B1'

In [50]:
project_dir = Path(f'/data/estorrs/mushroom/data/projects/submission_v1/{case}')
imaris_dir = project_dir / 'imaris' / 'rois'
imaris_dir_old = project_dir / 'imaris' / 'rois_old'

In [51]:
fig_dir = project_dir / 'figures' 
fig_dir.mkdir(parents=True, exist_ok=True)

In [52]:
config = yaml.safe_load(open(os.path.join(project_dir, 'registered', 'metadata.yaml')))
config = alter_filesystem(config, source_root, target_root)
config

{'resolution': 1.0,
 'sections': [{'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/submission_v1/HT206B1/registered/s0_HT206B1-U1_xenium.h5ad'}],
   'position': 0,
   'sid': 'HT206B1-U1'},
  {'data': [{'dtype': 'he',
     'filepath': '/data/estorrs/mushroom/data/projects/submission_v1/HT206B1/registered/s2_HT206B1-U4_he.tif'}],
   'position': 5,
   'sid': 'HT206B1-U4'},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/submission_v1/HT206B1/registered/s1_HT206B1-U2_multiplex.ome.tiff'}],
   'position': 15,
   'sid': 'HT206B1-U2'},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/submission_v1/HT206B1/registered/s3_HT206B1-U5_multiplex.ome.tiff'}],
   'position': 20,
   'sid': 'HT206B1-U5'},
  {'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/submission_v1/HT206B1/registered/s4_HT206B1-U8_xenium.h5ad'}],
   'position': 40,
   'sid': 'HT2

In [53]:
sid_to_z = {entry['sid']:i for i, entry in enumerate(config['sections'])}

In [54]:
case_to_rois = {
    'HT891Z1': {
        'roi1': (1300, 2100, 2500, 3500), # normal -> gp3
        'roi2': (4650, 5650, 1900, 2900), # normal -> gp3
        'roi2-p2': (4650, 5650, 1900, 2900), # PIN -> gp3
    },
    'HT913Z1': {
        'roi1': (4000, 4750, 5800, 7200), # normal -> gp3
        'roi1-p2': (4000, 4750, 5800, 7200) # normal -> gp3
    },
    'HT704B1': {
        'roi1': (2400, 3000, 5800, 6600), # DCIS -> IDC
        'roi2': (3700, 4300, 5750, 6500), # DCIS -> partial IDC
        'roi3': (0, 7250, 0, 9063), # full region trace of dcis
    },
    'HT206B1': {
        'roi1': (1650, 2600, 4900, 5750), # dcis -> idc
        'roi2': (2700, 4300, 3600, 5500) # dcis -> idc
    },
}

In [63]:
roi = 'roi2'

In [64]:
stacked_old = tifffile.imread(imaris_dir_old / f'{roi}.ome.tif')
stacked = tifffile.imread(imaris_dir / f'{roi}.ome.tif')
stacked.shape, stacked_old.shape

((16, 3, 1600, 1900), (15, 3, 1600, 1900))

In [65]:
sids, sids_old = [], []
for i, section in enumerate(config['sections']):
    sid = section['sid']
    # these dtypes are initially in there
    for entry in section['data']:
        dtype = entry['dtype']
        if dtype in ['he', 'batch2_he', 'xenium', 'multiplex']:
            sids_old.append(sid)
        if dtype in ['he', 'batch2_he', 'xenium', 'multiplex', 'cosmx', 'vishd']:
            sids.append(sid)
len(sids), len(sids_old)

(16, 15)

In [66]:
assert len(stacked) == len(sids)
assert len(stacked_old) == len(sids_old)

In [67]:
geojson_fp = imaris_dir_old / f'{roi}.geojson'
assert geojson_fp.exists()
d_old = json.load(open(geojson_fp))
# d_old

In [68]:
d = deepcopy(d_old)
for i, x in enumerate(d_old['features']):
    z_old = x['geometry']['plane']['z'] if 'plane' in x['geometry'] else 0

    sid_old = sids_old[z_old]
    
    z = sids.index(sid_old)
    
    if 'plane' in x['geometry']:
        d['features'][i]['geometry']['plane']['z'] = z
    

In [69]:
json.dump(d, open(imaris_dir / f'{roi}.geojson', 'w'))